# Lab 3: Sequence Alignment in Bioinformatics

- Name: AbdelRahman Adel AbdelFattah
- ID: 17012296


## Objective

Introduce students to sequence alignment techniques using the R programming language in bioinformatics. By the end of this lab, leverage built-in R libraries for the tasks.


## Prerequisites
- Basic knowledge in programming and R syntax.
-  Installed R, RStudio, and Jupyter Notebook with R kernel (IRkernel) on either a local environment or Google Colab.
-  Familiarity with sequence databases, such as GenBank or EMBL

## Part 1: Data Retrieval and Preprocessing

### Task 1.1: Retrieve Sequences

1.  Install and load the necessary package.

In [1]:
install.packages("rentrez")
install.packages("BiocManager")
BiocManager::install("Biostrings")

Installing package into '/opt/homebrew/lib/R/4.3/site-library'
(as 'lib' is unspecified)



Installing package into '/opt/homebrew/lib/R/4.3/site-library'
(as 'lib' is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.1 (2023-06-16)

Warning message:
"package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'Biostrings'"
Old packages: 'markdown', 'tinytex', 'utf8', 'vctrs', 'KernSmooth', 'Matrix',
  'foreign', 'lattice', 'mgcv', 'nlme', 'rpart', 'spatial', 'survival'



In [2]:
library(rentrez)
library(Biostrings)

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The following object is masked from 'package:utils':

    findMatches


The following objects are masked from 'package:base':

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attachin

2. Fetch a sequence from GenBank.

In [3]:
db <- "nuccore"
term <- "random"
ids<-entrez_search(db=db, term=term, retmax=250)$ids

In [4]:
raw_seqs<-entrez_fetch(db =db, id=ids, rettype = "fasta", retmode = "text")

3. Store the sequence in an appropriate R data structure.

In [5]:
file_name <- "seqs_1.fasta"
writeLines(raw_seqs, file_name)
seqs_1<-readDNAStringSet(file_name)
head(seqs_1)

DNAStringSet object of length 6:
    width seq                                               names               
[1] 22439 CACATTCGATGACAGTTAGACAG...AATGAAATGTATGTTTATAAAAC NW_003519799.1 Di...
[2]  1812 ATGGGAAATTATATATCCAAAGC...AATAAAAACAAAATTATTTTATT XM_003293730.1 Di...
[3] 48732 TTCCCCCACTCCCACACCCACCC...TTTTAATTTTTTAATTTTTTACT NW_003519586.1 Di...
[4]  1119 ATGGATAATAGTGCAAATGTAGC...TAGAAATTAAAACAAATAATTAA XM_003290906.1 Di...
[5] 66772 AATTTTAATTAATTATAAATAAA...TTTAAAAATAAAATTTTTTTTTT NW_003519494.1 Di...
[6]  2224 ATGTTTATCTTCTTTTTTAATCA...TTTTAATAGAGGAAAATAATTAT XM_003288976.1 Di...

### Task 1.2: Sequence Preprocessing

1. Identify sequences with gaps or ambiguous bases.

In [6]:
alpha_freq <- alphabetFrequency(seqs_1)
ambig<-alpha_freq[alpha_freq[,"N"] > 0,]
gaps<-alpha_freq[alpha_freq[,"-"] > 0,]

alpha_freq
ambig
gaps

A,C,G,T,M,R,W,S,Y,K,V,H,D,B,N,-,+,.
8514,2790,2828,8307,0,0,0,0,0,0,0,0,0,0,0,0,0,0
706,256,287,563,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18624,5659,5819,18342,0,0,0,0,0,0,0,0,0,0,288,0,0,0
463,139,179,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25649,8277,8011,24835,0,0,0,0,0,0,0,0,0,0,0,0,0,0
711,401,386,726,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29741,10724,10979,31291,0,0,0,0,0,0,0,0,0,0,0,0,0,0
596,178,202,366,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31089,10547,9988,31739,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1014,407,344,735,0,0,0,0,0,0,0,0,0,0,0,0,0,0


A,C,G,T,M,R,W,S,Y,K,V,H,D,B,N,-,+,.
18624,5659,5819,18342,0,0,0,0,0,0,0,0,0,0,288,0,0,0
32988,9532,9942,31608,0,0,0,0,0,0,0,0,0,0,1358,0,0,0
35094,11093,11796,35747,0,0,0,0,0,0,0,0,0,0,261,0,0,0
35992,11756,12277,36319,0,0,0,0,0,0,0,0,0,0,300,0,0,0
47195,14905,15625,48841,0,0,0,0,0,0,0,0,0,0,960,0,0,0
50030,16913,15660,49085,0,0,0,0,0,0,0,0,0,0,219,0,0,0
25,14,22,21,0,0,0,0,0,0,0,0,0,0,1,0,0,0
25,14,22,21,0,0,0,0,0,0,0,0,0,0,1,0,0,0
464339,325320,325569,457531,0,0,0,0,0,0,0,0,0,0,300000,0,0,0
87457,41262,59207,81556,0,0,0,0,0,0,0,0,0,0,11357,0,0,0


A,C,G,T,M,R,W,S,Y,K,V,H,D,B,N,-,+,.


2.  Handle sequences with gaps or ambiguous bases.

In [7]:
temp <- seqs_1
for(i in seq_len(nrow(alpha_freq))){
    if(alpha_freq[i,"-"] != 0){
        temp[i] <- gsub("-", "", temp[i])
    }
}
seqs_1_no_gaps <- temp
head(alphabetFrequency(seqs_1_no_gaps))
head(alphabetFrequency(seqs_1))

A,C,G,T,M,R,W,S,Y,K,V,H,D,B,N,-,+,.
8514,2790,2828,8307,0,0,0,0,0,0,0,0,0,0,0,0,0,0
706,256,287,563,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18624,5659,5819,18342,0,0,0,0,0,0,0,0,0,0,288,0,0,0
463,139,179,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25649,8277,8011,24835,0,0,0,0,0,0,0,0,0,0,0,0,0,0
711,401,386,726,0,0,0,0,0,0,0,0,0,0,0,0,0,0


A,C,G,T,M,R,W,S,Y,K,V,H,D,B,N,-,+,.
8514,2790,2828,8307,0,0,0,0,0,0,0,0,0,0,0,0,0,0
706,256,287,563,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18624,5659,5819,18342,0,0,0,0,0,0,0,0,0,0,288,0,0,0
463,139,179,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25649,8277,8011,24835,0,0,0,0,0,0,0,0,0,0,0,0,0,0
711,401,386,726,0,0,0,0,0,0,0,0,0,0,0,0,0,0


3. Convert all sequences to uppercase.

In [8]:
# Using readDNAStringSet makes sure that all sequences are converted to uppercase
head(seqs_1) == toupper(head(seqs_1))

[1] TRUE TRUE TRUE TRUE TRUE TRUE

## Part 2: Using Built-in Libraries for Sequence Alignment

Use at least the below Sequences in your execution:
1. Sequence A: AGCTGAACTAGCTAGCTGACTGACTGACTAGCTAGCTGACTAGCTG
2. Sequence B: AGCGAACTAGCTGACTGACGACTGACTAGCTGACTAGCTGACTAGC

In [9]:
seq_a <- DNAString("AGCTGAACTAGCTAGCTGACTGACTGACTAGCTAGCTGACTAGCTG")
seq_b <- DNAString("AGCGAACTAGCTGACTGACGACTGACTAGCTGACTAGCTGACTAGC")

### Task 2.1: Pairwise Sequence Alignment with Biostrings

1.  Use the ‘pairwiseAlignment’ function from the Biostrings package to perform alignments.

In [10]:
mat <- nucleotideSubstitutionMatrix(match = 1, mismatch = -1)
result<-pairwiseAlignment(seq_a, seq_b, type="global", substitutionMatrix = mat, gapOpening = -1, gapExtension = -1)
result

Global PairwiseAlignmentsSingleSubject (1 of 1)
pattern: AGCTGAACTAGCTAGCTGACTGACTGACTAGCT----AGCTGACTAGCTG
subject: AGC-GAACTAGCTGACTGAC-GACTGACTAGCTGACTAGCTGACTAGC--
score: 26 

2.  Compare the results (aligned sequences, scores) with those from custom implementations.

In [11]:
# using the custom implementation of Needleman-Wunsch algorithm from this url https://bioboot.github.io/bimm143_W20/class-material/nw/
seq_a_custom <- DNAString("AGCTGAACTAGCT-AGCTGAC-")
seq_a_custom <- DNAString("AGC-GAACTAGCTGA-CTGACG")
score_custom <- 14
mat_custom <- nucleotideSubstitutionMatrix(match = 1, mismatch = -1)
gapOpening_custom <- -1
gapExtension_custom <- -1

## Part 3: Jupyter Notebook Report

- Introduction: Use a Markdown cell to introduce the objective of this Notebook.
- Methodology: Explain the methods and libraries used in this Notebook.
- Results: All the code cells from Parts 1 to 4 serve as the results.
- Discussion: Use a Markdown cell to discuss the observations and interpretations of the tasks and results.
- Conclusion: Summarize the entire assignment and the findings in a Markdown cell.

#### Methodology

Basic data fetching using rentrez, and using Biostings to manipulate DNA sequences, using 2 libraries (rentrez, Biostrings)

#### Results

Biostrings make it easy to manipulate and handle the usual DNA sequences including gaps and ambiguous DNA sequences, and pairwiseAlignment creates the most optimal alignment according to a substitution matrix.